# Imports and general definitions
At the very beginning, we import necessary packages and make definitions regarding the resolution and the endpoint to the api.

In [ ]:
from datetime import datetime
import os
import requests
import json
from rasterio.transform import from_origin
from pyproj import CRS
from subprocess import Popen
import numpy as np
import gc
import subprocess as sp
import xarray as xa
import rioxarray as riox
import m3u8

ENDPOINT = "https://dunia.esa.int/api/streaming/band_metadata"
ENDPOINT_TILE_BOUNDS = "https://dunia.esa.int/api/streaming/s2-tiles/utm-bounds"
FPS = 25

S2_API = '/{tile_id}/{band}/{year}'

# Setting Parameters

Secondly, we set the parameters for the stream selection. We specify the following parameters:
- A Sentinel 2 tile id in *tile_id*
- The band of the stream. Possible values are B01, B02, B03, B04, B05, B06, B07, B08, B8A, B09, B11, B12
- The year of the observed data in *year*, starting from 2019.
- A start date and time within that year
- An end date and time within that year
- A path to store the geotiffs in.

In [ ]:
# tile
tile_id = '29PKR'

# band from B01, B02, B03, B04, B05, B06, B07, B08, B8A, B09, B11, B12
band = 'B09'

if band == 'B01' or band == 'B09':
    resolution = 60
    width = 1830
    height = 1830
elif band == 'B05' or band == 'B06' or band == 'B07' or band == 'B8A' or band == 'B11' or band == 'B12':
    resolution = 20
    width = 5490
    height = 5490
elif band == 'B02' or band == 'B03' or band == 'B04' or band == 'B08':
    resolution = 10
    width = 10980
    height = 10980
else:
    raise ValueError("Band not supported")

# year from [2019, 2020, 2021, 2022, 2023, 2024, 2025]
year = 2025

# start_date
start_date = datetime(year=year, month=3, day=1, hour=0, minute=0, second=0)

# end_date
end_date = datetime(year=year, month=3, day=30, hour=23, minute=59, second=59)

# path
save_path = './geotiffs/s2_bands/'

# if the path doesn't exist yet, it is made.
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)

# Accessing the stream
To receive an uri to the stream with the right resolution, we send a get request to the dunia api. This also returns metadata for each image in the stream.
The metadata contains:
- the sensing time of the image
- the frame number in the stream
- the timing of each frame in the stream
- the processing baseline

The variables stream_link and frames_metadata will be used in the oncoming steps. 

Cloud / No data based filtering needs to be applied in combination with the corresponding TCI stream.

In [ ]:
stream_link = None
frames_metadata = None

try:
    # load streams and metadata
    response = requests.get(url=ENDPOINT + S2_API.format(year=year, tile_id=tile_id, band=band),)
    print('Response code:', response.status_code)
    

    # saving the uri of the stream according to the selected resolution.
    stream_link = json.loads(response.content)['metadata']['stream_full']

    # saving metadata about the images:
    #   sensing time (sensing_time)
    #   frame number (image_number)
    # percentage of non-data in the image (nodata_pixel_percentage)
    # cloud coverage (cloudy_pixel_percentage)
    # filtering by cloud coverage and non-data percentage needs to be done
    # by loading metadata for same time and tile_id.
    frames_metadata = json.loads(response.content)['images']
except KeyError as e:
    print('Stream not found')
    

# Finding the frames
With the help of metadata of the whole stream we find segments in range of the specified dates.

In [ ]:

def round_to_two_decimal_places(value):
    return round(value, 2)


def find_segments_in_range(playlist_url, start, end):
    playlist = m3u8.load(playlist_url) 
    accumulated_duration = 1.4
    segments_in_range = []
    frame_rate = 25.0
    if start == end:
        end = end + 0.01
    

    for segment in playlist.segments:
        segment_start = accumulated_duration
        segment_end = round_to_two_decimal_places(accumulated_duration + segment.duration)

        if segment_start < end and segment_end > start:
            # The index of the first frame of the segment (typically 0)
            start_frame_index = round((segment_start - 1.4) * frame_rate)
            # The index of the last frame of the segment (typically 3)
            end_frame_index = round((segment_end - 1.4) * frame_rate) - 1

            relative_start_frame_index = (
                round((start - segment_start) * frame_rate)
                if start > segment_start
                else 0
            )

            relative_end_frame_index = (
                round((end - segment_start) * frame_rate)
                if end < segment_end
                else round((segment_end - segment_start) * frame_rate) - 1
            )

            frame_count = round(segment.duration * frame_rate)

            segments_in_range.append((
                segment.uri,
                (segment_start, segment_end),
                (start_frame_index, end_frame_index),
                (relative_start_frame_index, relative_end_frame_index),
                frame_count,
                segment.duration,
            ))

        accumulated_duration = segment_end

        if accumulated_duration >= end:
            break

    return segments_in_range

filtered_frames_metadata = []
# filtering the metadata according to the selected dates
for frame in frames_metadata:
    sensing_time = datetime.strptime(frame['sensing_time'], '%Y-%m-%dT%H:%M:%S.%f')
    if start_date <= sensing_time <= end_date:
        filtered_frames_metadata.append(frame)

start_pts = filtered_frames_metadata[0]['pts_time']
end_pts = filtered_frames_metadata[-1]['pts_time']

segments = find_segments_in_range(stream_link, start_pts, end_pts)

# Transform and CRS

Since we want to reference the image geologically, we look up the bounding box of the Sentinel 2 tile and find the crs. The transform to the bounding box and the crs are saved and will be applied in the last step. To get the correct bounding box and epsg the api is contacted again.

In [ ]:

try:
    # load tile bounds and epsg
    response = requests.get(url=ENDPOINT_TILE_BOUNDS + f"/{tile_id}")
    print('Response code:', response.status_code)

    response_content = json.loads(response.content.decode('utf-8'))

    epsg = response_content['utm']['epsg']
    bbox = response_content['utm']['bbox']
    
except KeyError as e:
    print('Tile bounds not found')
    

# saving the transform and the crs to apply to the extracted frames later
transform = from_origin(bbox[0], bbox[3], resolution, resolution)
crs = CRS.from_string(epsg)


# Creating Datasets and extracting to geotiffs

With the preparation done, we can finally get to the data. This step is done in one for loop. New images via ffmpeg are loaded and then rescaled with metadata to its actual values.
 
After the image is read into a DataArray the CRS is applied and a transform is made to complete geo-referencing the image.

Then we save the image- and geo-information in the DataArray to a geotiff.



In [ ]:
frame_count = 0

print('Loading and converting images:')

for segment in segments:
    segment_uri = segment[0]
    relative_start_frame_index = segment[3][0]
    relative_end_frame_index = segment[3][1]
    
     # Step 1: Stream the .ts file into memory
    with requests.get(segment_uri, stream=True) as response:
        response.raise_for_status()  # Ensure the request was successful
        ts_data = response.content  # Download the entire .ts file into memory

        # Step 2: Pass the streamed .ts file to ffmpeg via stdin
        for frame_index in range(relative_start_frame_index, relative_end_frame_index + 1):
            command = [
                'ffmpeg', '-hide_banner',
                '-i', 'pipe:0',  # Read input from stdin
                '-vf', f'select=eq(n\\,{frame_index})',
                '-pix_fmt', 'gray16le',
                '-f', 'rawvideo',
                'pipe:'
            ]
            
            # Run ffmpeg and pass the streamed .ts data via stdin
            process = sp.Popen(
                command,
                stdin=sp.PIPE,
                stdout=sp.PIPE,
                stderr=sp.PIPE
            )
            frame_data, error = process.communicate(input=ts_data)

            if process.returncode != 0:
                raise Exception(f"Error processing frame {frame_index}: {error.decode('utf-8')}")
                
            pix_dtype = np.uint16
            bytes_per_pixel_per_band = np.dtype(pix_dtype).itemsize

            # Ensure the buffer size matches the expected frame size
            expected_size = width * height * bytes_per_pixel_per_band
            if len(frame_data) != expected_size:
                print(f"Error: Frame {frame_index} size mismatch. Expected {expected_size}, got {len(frame_data)}")
                break

             # Convert the raw frame data to a NumPy array
            np_from_buffer = np.frombuffer(frame_data, dtype=pix_dtype)
            img_h_w = np_from_buffer.reshape((width, height))  # Assuming a single-band grayscale image
            print(".", end="")
            
            current_frame_metadata = filtered_frames_metadata[frame_count]
            scale = current_frame_metadata['scale_factor']
            offset = current_frame_metadata['add_offset']
            
            rescaled_img_h_w = ((img_h_w >> 4) * scale + offset).astype(np.uint16)
                        
            da_image = xa.DataArray(rescaled_img_h_w, dims=['y', 'x'])
    
            # # Apply the crs of the selected tile to the Dataset
            da_image.rio.write_crs(input_crs=crs, inplace=True)
            #
            # # Apply the transform to the bounding box of the tile to the Dataset
            da_image.rio.write_transform(transform=transform, inplace=True)

            # your code

            
            
            sensing_time = current_frame_metadata['sensing_time'][:19].replace(':', '').replace('-', '')

            # Save the image as a geotiff
            tif_name = f"{save_path}"
            tif_name += f"T{tile_id}_"
            tif_name += f"{sensing_time}_"
            tif_name += f"{current_frame_metadata['baseline']}_"
            tif_name += f"{band}_"
            tif_name += f"{resolution}m"
            tif_name += ".tif"

            da_image.rio.to_raster(tif_name)
        
            frame_count += 1
print("")
print("All done.")